## Dynamic Data Loading With AutoLoaders

In [0]:
dbutils.widgets.text("folder_file", "")

folder_file = dbutils.widgets.get("folder_file")

## Incremental Data Ingestion With Spark Streaming

In [0]:
df_flights = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "csv")\
    .option("cloudFiles.schemaLocation", f"abfss://bronze@dbdatalakejess.dfs.core.windows.net/{folder_file}/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode", "rescue")\
    .load(f"abfss://source@dbdatalakejess.dfs.core.windows.net/{folder_file}")

In [0]:
df_flights.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("checkpointLocation", f"abfss://bronze@dbdatalakejess.dfs.core.windows.net/{folder_file}/checkpoint")\
    .option("path", f"abfss://bronze@dbdatalakejess.dfs.core.windows.net/{folder_file}")\
    .start()